# Vehicle Detection Project

The goals / steps of this project are the following:

* Define functions to compute input features from dataset car/non-car images.
    - Color space change, Spatial binning, Color histogram, HOG with subsampling
* Combine (concatenate) input features, normalize/scale (using sklearn StandardScaler) them.
* Using a balanced dataset and shuffled train/test splits, train a single or ensemble of classifiers to detect cars.
    - Linear SVM, Decision trees, Deep neural networks
* Implement a generic sliding window technique that feeds patches from a given image to classifier for classification.
    - Use multiple window sizes
    - Come up with efficient scan strategy (e.g. only bottom half of image)
* Implement heatmap thresholding to combine multiple detections and remove false positives
* Use information from multiple frames to remove spurious false positives that appear in one or small number of frames.
    - Calculate motion trajectory if required


In [ ]:
import numpy as np
import cv2
import glob
import os
import random
import pickle
import time
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from scipy.ndimage.measurements import label
%matplotlib inline
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [ ]:
### Helper Functions

def readImage(imfile):
    img = cv2.imread(imfile)
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

def plotImageFiles(filenames):
    for f in filenames:
        fig = plt.figure()
        #fig.set_size_inches(3,6)
        img = readImage(f)
        plt.title(f)
        plt.axis('off')
        plt.imshow(img)
        
def plotImage(img, title=None, cmap=None):
    fig = plt.figure()
    #fig.set_size_inches(4,8)
    if title is not None:
        plt.title(title)
    #plt.axis('off')
    if cmap is None:
        plt.imshow(img)
    else:
        plt.imshow(img, cmap=cmap)
    
def plotMultipleImages(images, labels=None, ptitle=None, cmap=None):
    """ This function will plot the images specified in a
    single plot.
    """
    numImages = len(images)
    ii = 1
    for img in images:
        fig = plt.figure()
        if labels is not None:
            plt.title(labels[ii-1], fontsize="xx-small")
        #plt.axis('off')
        if cmap is not None:
            plt.imshow(img.squeeze(), cmap=cmap)
        else:
            plt.imshow(img.squeeze())
        ii += 1

def plotRectangles(img, bboxes, color=(0, 0, 255), thick=6, title=None):
    # Make a copy of the image
    imcopy = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
    plt.figure()
    if title is not None:
        plt.title(title)
    plt.imshow(imcopy)
    
print("*** Helper Functions Defined ***")

### Compute Input Features

In [ ]:
def convertImageColorSpace(img, color_space):
    """ Convenient wrapper over opencv cvtColor.
    """
    # Apply color conversion if other than 'RGB'
    if color_space != 'RGB':
        if color_space == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif color_space == 'LUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif color_space == 'HLS':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif color_space == 'YUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    else:
        feature_image = np.copy(img)
    return feature_image
        

def getBinSpatialFeatures(img, size=(32, 32)):
    """ Computes spatially binned color features
    """
    features = cv2.resize(img, size).ravel() 
    return features

def getColorHistogramFeatures(img, nbins=32, bins_range=(0, 256)):
    """ Computes color histogram of features for each image channel
    and combines them into a single feature vector.
    """
    channelhists = []
    for i in range(0, img.shape[2]):
        channelhist = np.histogram(img[:,:,i], bins=nbins, range=bins_range)
        channelhists.append(channelhist[0])
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate(channelhists)
    return hist_features

def getHOGFeatures(img, orient, pix_per_cell, cell_per_block, vis=False, feature_vec=True):
    """ Computes and returns HOG features and visualization (optional).
    NOTE: Hog automatically does (100/cell_per_block)% overlap between
    blocks.
    """
    if vis == True:
        features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=True, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    else:      
        features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=True, 
                       visualise=vis, feature_vector=feature_vec)
        return features
    
def extractImageFeatures(img, params):    
    """ Returns a combined image feature vector for given single image.
    """
    #1) Define an empty list to receive features
    img_features = []
    #2) Apply color conversion if other than 'RGB'
    feature_image = convertImageColorSpace(img, params['color_space'])
    #3) Compute spatial features if flag is set
    if params['spatial_feat'] == True:
        spatial_features = getBinSpatialFeatures(feature_image, size=params['spatial_size'])
        #4) Append features to list
        img_features.append(spatial_features)
    #5) Compute histogram features if flag is set
    if params['hist_feat'] == True:
        hist_features = getColorHistogramFeatures(feature_image, nbins=params['hist_bins'])
        #6) Append features to list
        img_features.append(hist_features)
    #7) Compute HOG features if flag is set
    if params['hog_feat'] == True:
        if params['hog_channel'] == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.extend(getHOGFeatures(feature_image[:,:,channel], 
                                    params['orient'], params['pix_per_cell'],
                                    params['cell_per_block'], vis=False, feature_vec=True))      
        else:
            hog_features = getHOGFeatures(feature_image[:,:,params['hog_channel']], params['orient'], 
                        params['pix_per_cell'], params['cell_per_block'], vis=False, feature_vec=True)
        #8) Append features to list
        img_features.append(hog_features)
    #9) Return concatenated array of features
    return np.concatenate(img_features)

def batchExtractImageFeatures(imgfiles, params):
    """ Returns list of combined image feature vectors for the given batch
    of images.
    """
    batch_features = []
    for imfile in imgfiles:
        img = readImage(imfile)
        features = extractImageFeatures(img, params)
        batch_features.append(features)
    return batch_features

def plotSampleHogImages():
    carfiles = glob.glob('data/vehicles/**/*.png', recursive=True)
    noncarfiles = glob.glob('data/non-vehicles/**/*.png', recursive=True)
    samples = min(len(carfiles), len(noncarfiles))
    ind = random.randint(0, samples-1)
    carimg = readImage(carfiles[ind])
    noncarimg = readImage(noncarfiles[ind])
    gscarimg = cv2.cvtColor(carimg, cv2.COLOR_RGB2GRAY)
    gsnoncarimg = cv2.cvtColor(noncarimg, cv2.COLOR_RGB2GRAY)
    _, carhogvis = getHOGFeatures(gscarimg, orient=9, pix_per_cell=4, cell_per_block=2, vis=True, feature_vec=True)
    _, noncarhogvis = getHOGFeatures(gsnoncarimg, orient=9, pix_per_cell=4, cell_per_block=2, vis=True, feature_vec=True)
    plotMultipleImages([carimg, carhogvis], cmap="gray")

#plotSampleHogImages()
    
print("*** Input Features Computation Functions Defined ***")

### Classifier Training

In [ ]:
def getClassifierParams():
    params = dict()
    params['color_space'] = 'YCrCb' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
    params['orient'] = 9  # HOG orientations
    params['pix_per_cell'] = 8 # HOG pixels per cell
    params['cell_per_block'] = 2 # HOG cells per block
    params['hog_channel'] = "ALL" # Can be 0, 1, 2, or "ALL"
    params['spatial_size'] = (32, 32) # Spatial binning dimensions
    params['hist_bins'] = 32    # Number of histogram bins
    params['spatial_feat'] = False # Spatial features on or off
    params['hist_feat'] = False # Histogram features on or off
    params['hog_feat'] = True # HOG features on or off
    return params
    

def getTrainedClassifier(datafile = "./classifier/svc.pickle", forcetrain=False):
    """ Loads a pre-trained classifier if available or trains a new one.
    It takes care of loading up the dataset images, extracting the image
    features, normalizing them and training the classifier and saving it
    if its the first time or forceTrain is True.
    """
    if os.path.isfile(datafile) and not forcetrain:
        print("--> Returning saved classifier")
        with open(datafile, "rb") as f:
            pickledata = pickle.load(f)
            clf = pickledata["classifier"]
            params = pickledata["params"]
            return clf, params
    carfiles = glob.glob('data/vehicles/**/*.png', recursive=True)
    noncarfiles = glob.glob('data/non-vehicles/**/*.png', recursive=True)
    numsamples = min(len(carfiles), len(noncarfiles))
    params = getClassifierParams()
    print("--> Extracting features. Numsamples = ", numsamples)
    carfeatures = batchExtractImageFeatures(carfiles[0:numsamples], params)
    noncarfeatures = batchExtractImageFeatures(noncarfiles[0:numsamples], params)
    X = np.vstack((carfeatures, noncarfeatures)).astype(np.float64)                        
    print("--> Scaling features")
    # Fit a per-column scaler
    X_scaler = StandardScaler().fit(X)
    # Apply the scaler to X
    scaled_X = X_scaler.transform(X)
    # Define the labels vector
    y = np.hstack((np.ones(len(carfeatures)), np.zeros(len(noncarfeatures))))
    # Split up data into randomized training and test sets
    rand_state = np.random.randint(0, 100)
    X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.2, random_state=rand_state)
    # Use a linear SVC 
    svc = LinearSVC()
    # Check the training time for the SVC
    print("--> Starting to Train Classifier")
    t=time.time()
    svc.fit(X_train, y_train)
    t2 = time.time()
    print("--> ", round(t2-t, 2), 'Seconds to train SVC...')
    # Check the score of the SVC
    print('--> Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
    print("--> Saving Classifier")    
    t=time.time()
    with open(datafile, "wb") as f:
        pickle.dump( { 'classifier': svc, 'params': params,}, f )
    return svc, params

def testClassifierOnDatasetImages():
    carfiles = glob.glob('data/vehicles/**/*.png', recursive=True)
    noncarfiles = glob.glob('data/non-vehicles/**/*.png', recursive=True)
    numsamples = min(len(carfiles), len(noncarfiles))
    svc, params = getTrainedClassifier()
    wrongcars = 0
    wrongnoncars = 0
    for i in range(0, numsamples):
        if i%1000 == 0:
            print("--> {}".format(i))
        carimg = readImage(carfiles[i])
        noncarimg = readImage(noncarfiles[i])
        carfeats = extractImageFeatures(carimg, params)
        noncarfeats = extractImageFeatures(noncarimg, params)
        pc = svc.predict(carfeats)
        pnc = svc.predict(noncarfeats)
        if pc != 1:
            wrongcars += 1
        if pnc != 0:
            wrongnoncars += 1
    print("wrongcars = {} or {}, wrongnoncars = {} or {}".format(wrongcars, (wrongcars*100.0)/numsamples,wrongnoncars, (wrongnoncars*100.0)/numsamples))

#testClassifierOnDatasetImages()
    
print("*** Classifier Training Functions Defined ***")

### Sliding Window Technique

In [ ]:
def plotSlidingWindows(img, scale, ystart=None, ystop=None):
    """ Debug function to plot sliding windows over image.
    """
    if ystart is None or ystop is None:
        #Select bottom half and snip out the hood
        ystart, ystop = img.shape[0]//2, int(img.shape[0] * 0.9)
    wimg = img[ystart:ystop,:,:]
    wimshape = wimg.shape
    if scale != 1:
        wimg = cv2.resize(wimg, (int(wimshape[1]/scale), int(wimshape[0]/scale)))
    winszxpix = winszypix = 64
    xsteps = wimg.shape[1] // winszxpix
    ysteps = wimg.shape[0] // winszypix
    bboxes = []
    for y in range(0, ysteps):
        for x in range(0, xsteps):
            tl = (x * winszxpix, y * winszypix)
            br = (x * winszxpix + winszxpix, y * winszypix + winszypix)
            bboxes.append((tl, br))
    plotRectangles(wimg, bboxes, title="Scale = " + str(scale))
    
def slidingWindowCarDetect(clf, img, scale, X_scaler, params, vis=True, ystart=None, ystop=None):
    """ This function slides a window of specified size over given image
    in x and y directions and computes features for each window and runs
    it through a classifier to determine if we've identified a car in the
    window.
    The window size is fixed but we can change the image size using the scale
    parameter. In essense, we fix the window size and change the canvas size
    to get multi-scale windows in which to search for cars.
    """
    car_bboxes = []
    if ystart is None or ystop is None:
        #Select bottom half and snip out the hood
        ystart, ystop = img.shape[0]//2, int(img.shape[0] * 0.9)
    wimg = convertImageColorSpace(img[ystart:ystop,:,:], params['color_space'])
    if vis:
        draw_img = np.copy(img)
    wimshape = wimg.shape
    if scale != 1:
        wimg = cv2.resize(wimg, (int(wimshape[1]/scale), int(wimshape[0]/scale)))
    if params['hog_feat'] == True:
        #Get HOG features for whole image on all channels
        #NOTE: Hog features for each channel will be of shape:
        #nblocksy, nblocksx, cell_per_block, cell_per_block, orient)
        hog_features = []
        if params['hog_channel'] == 'ALL':
            for channel in range(wimg.shape[2]):
                hf = getHOGFeatures(wimg[:,:,channel], params['orient'], params['pix_per_cell'],
                                    params['cell_per_block'], vis=False, feature_vec=False)
                hog_features.append(hf)
        else:
            hf = getHOGFeatures(wimg[:,:,params['hog_channel']], params['orient'], params['pix_per_cell'],
                                params['cell_per_block'], vis=False, feature_vec=False)
            hog_features.append(hf)
    # Define blocks and steps as above
    nxblocks = (wimg.shape[1] // params['pix_per_cell']) - 1
    nyblocks = (wimg.shape[0] // params['pix_per_cell']) - 1 
    nfeat_per_block = params['orient'] * params['cell_per_block'] ** 2
    # 64 was the orginal training set img size
    window = 64
    nblocks_per_window = (window // params['pix_per_cell']) - 1
    blocks_per_step = 2  # Instead of overlap, define how many blocks to step
    nxsteps = (nxblocks - nblocks_per_window) // blocks_per_step
    nysteps = (nyblocks - nblocks_per_window) // blocks_per_step
    for yb in range(nysteps):
        for xb in range(nxsteps):
            patch_features = []
            ypos = yb * blocks_per_step
            xpos = xb * blocks_per_step
            xleft = xpos * params['pix_per_cell']
            ytop = ypos * params['pix_per_cell']
            if params['hist_feat'] == True or params['spatial_feat'] == True:    
                # Extract the image patch and size it down to training set image size
                subimg = cv2.resize(wimg[ytop:ytop+window, xleft:xleft+window], (64,64))
            if params['spatial_feat'] == True:
                # Get color features
                spatial_features = getBinSpatialFeatures(subimg, size=params['spatial_size'])
                patch_features.append(spatial_features)
            if params['hist_feat'] == True:
                hist_features = getColorHistogramFeatures(subimg, nbins=params['hist_bins'])
                patch_features.append(hist_features)
            if params['hog_feat'] == True:
                # Extract HOG for this patch
                patch_hog_features = []
                for hogch in hog_features:
                    hogf = hogch[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel()
                    patch_hog_features.append(hogf)
                patch_hog_features = np.hstack(patch_hog_features)
                patch_features.append(patch_hog_features)

            # Scale features and make a prediction
            test_features = X_scaler.fit_transform(np.hstack(patch_features).reshape(-1))
            #test_features = X_scaler.transform(np.hstack((shape_feat, hist_feat)).reshape(1, -1))    
            test_prediction = clf.predict(test_features)
            #print("test_prediction = ", test_prediction)
            
            if test_prediction == 1:
                xbox_left = np.int(xleft * scale)
                ytop_draw = np.int(ytop * scale)
                win_draw = np.int(window * scale)
                tl = (xbox_left, ytop_draw + ystart)
                br = (xbox_left + win_draw, ytop_draw + win_draw + ystart)
                if vis:
                    cv2.rectangle(draw_img, tl, br, (0,0,255), 6)
                car_bboxes.append((tl, br))
    if vis:     
        return car_bboxes, draw_img
    else:
        return car_bboxes
    
print("*** Sliding Window Technique Functions Defined ***")


### Combining Overlaps and False Positives Removal

In [ ]:
def addHeat(heatmap, bbox_list):
    """ Adds a vote to all car pixels. It works based on the
    fact that multiple sliding windows will overlap on a real
    car vs. false positives. Therefore, pixels that truly belong
    to a car will have higher votes and we threshold them off.
    """
    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1

    # Return updated heatmap
    return heatmap# Iterate through list of bboxes

def applyHeatThreshold(heatmap, threshold):
    # Zero out pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    # Return thresholded map
    return heatmap

def drawLabeledBboxes(img, labels):
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        # Draw the box on the image
        cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 6)
    # Return the image
    return img

print("*** Overlap Combining and False Positives Removal Functions Defined ***")


### Overall Pipeline

In [ ]:
def pipeline(img, X_scaler, inter_vis_enable=False):
    clf, params = getTrainedClassifier()
    car_bboxes = slidingWindowCarDetect(clf, img, 1.25, X_scaler, params, vis=inter_vis_enable)
    #car_bboxes, draw_img = slidingWindowCarDetect(clf, img, 1.5, X_scaler, params, vis=vis)
    #plotImage(draw_img)
    heatmap = np.zeros_like(img[:,:,0]).astype(np.float)
    heatmap = addHeat(heatmap, car_bboxes)
    heatmap = applyHeatThreshold(heatmap, 1)
    heatmap = np.clip(heatmap, 0, 255)
    labels = label(heatmap)
    dimg = drawLabeledBboxes(np.copy(img), labels)
    return dimg

In [ ]:
def testPipelineOnTestImages():
    tfiles = glob.glob('test_images/test*.jpg')
    X_scaler = StandardScaler()
    for tfile in tfiles:
        timg = readImage(tfile)
        dimg = pipeline(timg, X_scaler)
        plotImage(dimg)

#testPipelineOnTestImages()


### Vehicle Detection on Video

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

#TODO: Basic pre-processing
X_scaler = StandardScaler()
def process_image(image):
    return pipeline(image, X_scaler)

In [ ]:
outvideofile = 'project_video_output.mp4'
pvideo = VideoFileClip("project_video.mp4")
ovideo = pvideo.fl_image(process_image)
%time ovideo.write_videofile(outvideofile, audio=False)

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(outvideofile))